In [376]:
from utils.random_nfa_generator import generate
from utils.heuristics import *

import numpy as np

from copy import copy as copy2

from queue import PriorityQueue
from random import shuffle

import networkx as nx
from FAdo.conversions import *
from FAdo.reex import *

from utils.fadomata import *


In [377]:
n = np.random.randint(3, 6)  # np.random.randint(3, maxN)
k = 5  # np.random.choice([2, 5, 10])
d = np.random.choice([0.2, 0.5])
gfa = generate(n, k, d, 'in-memory')


In [394]:
def is_epsilon(re):
    if str(re) == '@epsilon':
        return True
    else:
        return False
    
def eliminate_new(gfa: GFA, st: int):
    """Eliminate a state.

    :param int st: state to be eliminated"""
    if st in gfa.delta and st in gfa.delta[st]:
        if type(gfa.delta[st][st]) == reex.CStar:
            r2 = copy2(gfa.delta[st][st])
        r2 = copy2(reex.CStar(gfa.delta[st][st], copy2(gfa.Sigma)))
        del gfa.delta[st][st]
    else:
        r2 = None
    for s in gfa.delta:
        if st not in gfa.delta[s]:
            continue
        r1 = copy2(gfa.delta[s][st])
        
        del gfa.delta[s][st]
        for s1 in gfa.delta[st]:
            r3 = copy2(gfa.delta[st][s1])
            if r2 is not None:
                if r1 == r2 or r2 == r3:
                    r = reex.CConcat(
                        r1, r3, copy2(gfa.Sigma))
                elif is_epsilon(r1):
                    if is_epsilon(r3):
                        r = r2
                    else:
                        r = reex.CConcat(r2, r3, copy2(gfa.Sigma))
                elif is_epsilon(r3):
                    r = reex.CConcat(r1, r2, copy2(gfa.Sigma))
                else:
                    r = reex.CConcat(r1, reex.CConcat(
                        r2, r3, copy2(gfa.Sigma)), copy2(gfa.Sigma))
            else:
                if is_epsilon(r1):
                    r = r3
                elif is_epsilon(r3):
                    r = r1
                else:
                    r = reex.CConcat(
                        r1, r3, copy2(gfa.Sigma))
                
            if s1 in gfa.delta[s]:
                if gfa.delta[s][s1] == r:
                    gfa.delta[s][s1] = r
                elif is_epsilon(r) and type(gfa.delta[s][s1]) == reex.CStar:
                    gfa.delta[s][s1] = gfa.delta[s][s1]
                elif is_epsilon(gfa.delta[s][s1]) and type(r) == reex.CStar:
                    gfa.delta[s][s1] = r
                else:
                    if str(gfa.delta[s][s1]) > str(r):
                        gfa.delta[s][s1] = reex.CDisj(
                            r, gfa.delta[s][s1], copy2(gfa.Sigma))
                    else:
                        gfa.delta[s][s1] = reex.CDisj(
                            gfa.delta[s][s1], r, copy2(gfa.Sigma))
            else:
                gfa.delta[s][s1] = r
    del gfa.delta[st]


def eliminate_by_state_weight_heuristic_new(gfa: GFA) -> RegExp:
    pq = PriorityQueue()
    for i in range(1, len(gfa.States) - 1):
        pq.put((get_weight(gfa, i), i))
    while not pq.empty():
        eliminate_new(gfa, pq.get()[1])
        # gfa.eliminate(pq.get()[1])
    if gfa.Initial in gfa.delta and gfa.Initial in gfa.delta[gfa.Initial]:
        return CConcat(CStar(gfa.delta[gfa.Initial][gfa.Initial]), gfa.delta[gfa.Initial][list(gfa.Final)[0]])
    else:
        return gfa.delta[gfa.Initial][list(gfa.Final)[0]]


In [395]:
gfa_dup = gfa.dup()
print(len(str(eliminate_by_state_weight_heuristic(gfa_dup))))
gfa_dup = gfa.dup()
print(len(str(eliminate_by_state_weight_heuristic_new(gfa_dup))))


119
r1:  4 + 3 r2:  4 1
r1:  (0 + 2) + 4 r2:  0 (2* ((4 + 3) + (4 1)))
r1:  @epsilon r2:  0 (2* 4)
r1:  @epsilon r2:  1 ((((0 + 2) + 4) + (0 (2* ((4 + 3) + (4 1)))))* ((0 (2* 4)) + @epsilon))
86


In [388]:
gfa_dup = gfa.dup()
r1 = eliminate_by_repeated_state_weight_heuristic(gfa_dup)

In [389]:
gfa_dup = gfa.dup()
r2 = eliminate_by_state_weight_heuristic_new(gfa_dup)


In [390]:
str(r1)

'(@epsilon @epsilon) + ((@epsilon 1) ((((0 + 2) + 4) + (0 (2* ((4 + 3) + (4 1)))))* (@epsilon + (0 (2* (4 @epsilon))))))'

In [391]:
str(r2)

'(1 ((((0 + 2) + 4) + (0 (2* ((4 + 3) + (4 1)))))* ((0 (2* 4)) + @epsilon))) + @epsilon'

In [385]:
r2

CDisj(CConcat(CAtom(1),CConcat(CStar(CDisj(CDisj(CDisj(CAtom(0),CAtom(2)),CAtom(4)),CConcat(CAtom(0),CConcat(CStar(CAtom(2)),CDisj(CDisj(CAtom(4),CAtom(3)),CConcat(CAtom(4),CAtom(1))))))),CDisj(CConcat(CAtom(0),CConcat(CStar(CAtom(2)),CAtom(4))),CEpsilon()))),CEpsilon())